# Binance Currency Pair Analysis



# 1. Introduction

The purpose of this analysis is to examine the trading activity of various currency pairs involving fiat currencies on Binance. We will start by retrieving the necessary data, filtering for relevant pairs, and then proceed to analyze the data, with a particular focus on trading volume in USD for key cryptocurrencies like BTC, ETH, and USDT. This analysis aims to provide insights into the trading patterns and market behavior on the Binance platform.

In this notebook, we will conduct a general evaluation of the data provided by the Binance API, focusing on currency pairs that involve fiat currencies. The goal is to identify and analyze the available pairs, as well as to detect any new fiat currencies that may have been added to the platform.

## 1.1 Loading Necessary Libraries
In this section, we will import the libraries required for making API requests, handling data, and managing files. These include:

In [1]:
import requests  # To handle HTTP requests to the Binance API
import pandas as pd  # To structure and manipulate the data obtained
import os  # To manage directories for saving the data

## 1.2 Setting Up Directories**
We will set up a base directory to store the data files downloaded from Binance. This ensures that all data is organized and easily accessible for further analysis.

In [2]:
# Definir la carpeta base donde se guardarán los archivos CSV
base_folder = 'datos_historicos_binance'

# Crear la carpeta base si no existe
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# 2. Data Retrieval and Download

## 2.1 Retrieving Information on Currency Pairs in Binance

In this section, we will retrieve information on all available currency pairs on Binance using their API. Our goal is to identify pairs that involve fiat currencies, as these will be the focus of our analysis.



In [3]:
# Lista completa de monedas fiat
fiat_currencies = [
    'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CNY', 'INR', 'RUB', 
    'BRL', 'ARS', 'NGN', 'TRY', 'MXN', 'ZAR', 'PLN', 'SEK', 'NOK', 'DKK', 
    'HKD', 'SGD', 'NZD', 'KRW', 'THB', 'MYR', 'IDR', 'PHP', 'VND', 'CZK', 
    'HUF', 'RON', 'BGN', 'HRK', 'RSD', 'UAH', 'KZT', 'AZN', 'GEL', 'BYN',
    'EGP', 'ILS', 'SAR', 'AED', 'QAR', 'KWD', 'BHD', 'OMR', 'TND', 'MAD', 
    'DZD', 'TWD', 'LKR', 'PKR', 'BDT', 'KES', 'TZS', 'UGX', 'GHS', 'XAF', 
    'XOF', 'XPF', 'MUR', 'SCR', 'MGA', 'ZMW', 'MWK', 'BWP', 'NAD', 'SLL',
    'LRD', 'LSL', 'SZL', 'MZN', 'BIF', 'CDF', 'DJF', 'ETB', 'GNF', 'HTG',
    'LRD', 'MGA', 'NPR', 'PGK', 'PGK', 'SHP', 'SLL', 'SOS', 'SSP', 'STD',
    'SYP', 'TOP', 'TTD', 'TWD', 'VUV', 'WST', 'XCD', 'YER', 'ZMW'
]

In [4]:
def obtener_info_pares_binance():
    # URL del endpoint de exchangeInfo de Binance
    url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    # Realizar la solicitud GET al endpoint
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir los datos de respuesta a formato JSON
        data = response.json()
        
        # Crear un diccionario para almacenar la información sobre los pares de divisas
        info_pares = {}
        
        # Iterar sobre los datos para obtener la información relevante sobre cada par de divisas
        for symbol_info in data['symbols']:
            symbol = symbol_info['symbol']
            base_currency = symbol_info['baseAsset']
            quote_currency = symbol_info['quoteAsset']
            
            # Almacenar la información en el diccionario
            info_pares[symbol] = {'base_currency': base_currency, 'quote_currency': quote_currency}
        
        return info_pares
    else:
        print(f"Error al obtener información sobre los pares de divisas de Binance. Código de estado: {response.status_code}")
        return None

# Obtener información sobre los pares de divisas
info_pares = obtener_info_pares_binance()

In [5]:
import igraph as ig
import plotly.graph_objects as go

# Suponiendo que 'info_pares' contiene el diccionario con la información de los pares de divisas
info_pares = obtener_info_pares_binance()

# Listas de monedas fiat para AEs y EMDEs
aes_fiat_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'CHF', 'CZK']
emdes_fiat_currencies = [
    'BRL', 'ARS', 'NGN', 'TRY', 'MXN', 'ZAR', 'PLN', 'RUB', 'RON', 'UAH'
]

# Colores
aes_color = '#AA322F'  # Rojo oscuro para AEs
emdes_color = '#6FA5E2'  # Azul pastel más oscuro para EMDEs
crypto_color = '#D3D3D3'  # Gris claro para criptomonedas
crypto_label_color = '#A9A9A9'  # Gris un poco más oscuro para etiquetas de criptomonedas

# Extraer los vertices y las aristas a partir de 'info_pares'
vertices = set()
edges = []

for symbol, details in info_pares.items():
    base_currency = details['base_currency']
    quote_currency = details['quote_currency']
    vertices.add(base_currency)
    vertices.add(quote_currency)
    edges.append((base_currency, quote_currency))

# Convertir el conjunto de vertices a una lista
vertices = list(vertices)

# Crear el grafo en igraph
G_ig = ig.Graph()
G_ig.add_vertices(vertices)
G_ig.add_edges(edges)

# Filtrar nodos con grado menor que 3
low_degree_nodes = [v.index for v in G_ig.vs if G_ig.degree(v) < 3]
G_ig.delete_vertices(low_degree_nodes)

# Recalcular el PageRank después de eliminar los nodos de bajo grado
pagerank = G_ig.pagerank()
max_pagerank = max(pagerank)

# Exagerar las diferencias aplicando una transformación exponencial al PageRank
min_size = 1  # Tamaño mínimo para nodos con solo 3 conexiones
mid_size = 10 # Tamaño para nodos con 4 conexiones
max_size = 40 # Tamaño máximo para nodos con más conexiones y alto PageRank

# Asignar tamaños basados en el número de conexiones y PageRank
vertex_sizes = []
for i in range(len(G_ig.vs)):
    degree = G_ig.degree(i)
    if degree == 3:
        vertex_sizes.append(min_size)
    elif degree == 4:
        vertex_sizes.append(mid_size)
    else:
        vertex_sizes.append(min_size + (pagerank[i] / max_pagerank) ** 0.3 * (max_size - min_size))

# Encontrar las criptomonedas con los 5 mayores valores de PageRank
top_5_cryptos = sorted(range(len(pagerank)), key=lambda i: pagerank[i], reverse=True)[:5]

# Asignar colores a los nodos y etiquetas: AEs, EMDEs y criptomonedas
vertex_colors = []
label_colors = []
for i in range(len(G_ig.vs)):
    if G_ig.vs[i]['name'] in aes_fiat_currencies:
        vertex_colors.append(aes_color)
        label_colors.append(aes_color)
    elif G_ig.vs[i]['name'] in emdes_fiat_currencies:
        vertex_colors.append(emdes_color)
        label_colors.append(emdes_color)
    else:
        vertex_colors.append(crypto_color)
        label_colors.append(crypto_label_color)

# Asignar forma a los nodos
vertex_shapes = [
    'triangle-up' if G_ig.vs[i]['name'] in aes_fiat_currencies or G_ig.vs[i]['name'] in emdes_fiat_currencies else 'circle'
    for i in range(len(G_ig.vs))
]

# Asignar etiquetas a las monedas fiat y a las criptomonedas en el top 5 de PageRank
vertex_labels = [
    G_ig.vs[i]['name'] if G_ig.vs[i]['name'] in aes_fiat_currencies or G_ig.vs[i]['name'] in emdes_fiat_currencies or i in top_5_cryptos else ''
    for i in range(len(G_ig.vs))
]

# Definir tamaños de las etiquetas basados en PageRank
min_font_size = 8
max_font_size = 18
vertex_label_sizes = [min_font_size + (pr / max_pagerank) ** 0.3 * (max_font_size - min_font_size) for pr in pagerank]

# Aplicar el layout de Fruchterman-Reingold
layout = G_ig.layout_fruchterman_reingold()

# Extraer las posiciones del layout y ajustar para aristas curvas
Xn = [layout[k][0] for k in range(len(G_ig.vs))]
Yn = [layout[k][1] for k in range(len(G_ig.vs))]
Xe = []
Ye = []
for edge in G_ig.es:
    x_mid = (layout[edge.source][0] + layout[edge.target][0]) / 2
    y_mid = (layout[edge.source][1] + layout[edge.target][1]) / 2
    Xe += [layout[edge.source][0], x_mid, layout[edge.target][0], None]
    Ye += [layout[edge.source][1], y_mid, layout[edge.target][1], None]

# Graficar usando Plotly
trace1 = go.Scatter(
    x=Xe,
    y=Ye,
    mode='lines',
    line=dict(color='rgb(240, 240, 240)', width=1),  # Líneas de color gris claro
    hoverinfo='none'
)

trace2 = go.Scatter(
    x=Xn,
    y=Yn,
    mode='markers+text',
    name='ntw',
    marker=dict(
        symbol=vertex_shapes,
        size=vertex_sizes, 
        color=vertex_colors,
        line=dict(width=0)  # Eliminar el borde de los nodos
    ),
    text=vertex_labels,
    textposition='top center',
    textfont=dict(color=label_colors, size=vertex_label_sizes),  # Usar tamaños de fuente basados en PageRank
    hoverinfo='text'
)

layout_plot = go.Layout(
    title='<b>Cryptocurrency-Fiat Exchange Network: A Visual Analysis of Market Interactions Across Economies<b>',
    title_font_color="#AA322F",  
    font=dict(size=12),
    showlegend=False,
    autosize=False,
    width=800,
    height=800,
    paper_bgcolor='white',  # Fondo blanco
    plot_bgcolor='white',   # Fondo blanco del área de trazado
    xaxis=dict(showline=False, zeroline=False, showgrid=False, showticklabels=False),
    yaxis=dict(showline=False, zeroline=False, showgrid=False, showticklabels=False),
    margin=dict(l=40, r=40, b=85, t=100),
    hovermode='closest'
)

fig = go.Figure(data=[trace1, trace2], layout=layout_plot)
fig.show()


## 2.2 Filtering Currency Pairs by Fiat Currencies

Once we have retrieved the currency pair information, we will filter the pairs to focus only on those that involve fiat currencies. This step is essential for narrowing down our analysis to the most relevant pairs.


In [6]:
# Filtrar para obtener solo los pares de interés con las monedas fiat
fiat_pairs = [pair for pair, currencies in info_pares.items() if currencies['quote_currency'] in fiat_currencies or currencies['base_currency'] in fiat_currencies]

print(f"Total pairs with fiat currencies: {len(fiat_pairs)}")


Total pairs with fiat currencies: 462


## 2.3 Downloading and Saving Historical Data for Fiat-Crypto Pairs on Binance

In this section, we will download historical daily data for the fiat-crypto pairs we've identified from Binance's API. The data will be saved in monthly CSV files, organized by fiat currency and cryptocurrency.

In [7]:
import requests
import pandas as pd
import os
from tqdm.notebook import tqdm

# Define the base folder where CSV files will be stored
base_folder = 'datos_historicos_binance'

# Create the base folder if it does not exist
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# Initialize a counter and a list to track saved files
total_files_saved = 0
saved_files = []

def obtener_datos_y_guardar_csv(symbol, base_currency, quote_currency):
    global total_files_saved
    global saved_files

    # Build the URL for the Binance API
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d'
    
    # Send a GET request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response data to JSON format
        data = response.json()
        
        # Create a pandas DataFrame with the data
        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 
            'close_time', 'quote_asset_volume', 'number_of_trades', 
            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
        ])
        
        # Convert the 'timestamp' column to datetime format
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Determine the fiat currency and crypto currency based on their position
        if base_currency in fiat_currencies:
            fiat_currency = base_currency
            crypto_currency = quote_currency
        else:
            fiat_currency = quote_currency
            crypto_currency = base_currency
        
        # Create a folder for the fiat currency if it doesn't exist
        fiat_folder_path = os.path.join(base_folder, fiat_currency)
        if not os.path.exists(fiat_folder_path):
            os.makedirs(fiat_folder_path)
        
        # Create a subfolder for the cryptocurrency if it doesn't exist
        crypto_folder_path = os.path.join(fiat_folder_path, crypto_currency)
        if not os.path.exists(crypto_folder_path):
            os.makedirs(crypto_folder_path)
        
        # Save the data into monthly CSV files with daily data inside
        for month, df_month in df.groupby(df['timestamp'].dt.to_period('M')):
            csv_file_path = os.path.join(crypto_folder_path, f'{fiat_currency.lower()}_{crypto_currency.lower()}_{month.start_time.strftime("%Y-%m")}.csv')
            df_month.to_csv(csv_file_path, index=False)
            total_files_saved += 1
            saved_files.append(csv_file_path)
    else:
        print(f"Error fetching data for {symbol}. Status code: {response.status_code}")

# Download data for all the pairs of interest with a progress bar
for pair in tqdm(fiat_pairs, desc="Processing pairs"):
    base_currency = info_pares[pair]['base_currency']
    quote_currency = info_pares[pair]['quote_currency']
    obtener_datos_y_guardar_csv(pair, base_currency, quote_currency)

# Summary message
print(f"Total files saved: {total_files_saved}")

Processing pairs:   0%|          | 0/462 [00:00<?, ?it/s]

Total files saved: 6099


## 2.3 Loading and Combining the Data for Analysis

In this section, we load the previously saved CSV files, which contain the historical data for each fiat-crypto pair. We then combine these data files into a single DataFrame to facilitate further analysis. This step is crucial for ensuring that our data is well-organized and ready for more detailed exploration.


In [8]:
import os
import pandas as pd
from tqdm.notebook import tqdm

# Initialize a list to store the DataFrames
dfs = []

# Get the list of fiat currencies to iterate over and initialize the progress bar
fiat_currencies = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]

# Iterate through the fiat currency folders with a progress bar
for fiat_currency in tqdm(fiat_currencies, desc="Processing fiat currencies"):
    fiat_folder_path = os.path.join(base_folder, fiat_currency)
    
    # Iterate through the crypto currency folders
    for crypto_currency in os.listdir(fiat_folder_path):
        crypto_folder_path = os.path.join(fiat_folder_path, crypto_currency)
        if os.path.isdir(crypto_folder_path):
            # Iterate through the CSV files
            for csv_file in os.listdir(crypto_folder_path):
                csv_file_path = os.path.join(crypto_folder_path, csv_file)
                if csv_file_path.endswith('.csv'):
                    # Create a pandas DataFrame from the CSV data
                    df = pd.read_csv(csv_file_path)
                    # Add columns for fiat and crypto currencies
                    df['fiat_currency'] = fiat_currency
                    df['crypto_currency'] = crypto_currency
                    # Append the DataFrame to the list
                    dfs.append(df)

# Combine all the DataFrames into one
final_df = pd.concat(dfs, ignore_index=True)

Processing fiat currencies:   0%|          | 0/15 [00:00<?, ?it/s]

Check if there are any missing values in the dataset

In [9]:
# Check if there are any missing values in the dataset
if final_df.isnull().sum().sum() == 0:
    print("Data check complete: No missing values. All data is clean and ready for analysis.")
else:
    print("Data check complete: There are missing values in the dataset. Further inspection is needed.")

# Basic descriptive statistics (can be optional or removed if not needed for output)
final_stats = final_df.describe()
print("Basic statistics generated. Data is ready for further analysis.")

Data check complete: No missing values. All data is clean and ready for analysis.
Basic statistics generated. Data is ready for further analysis.


# 3. Data Analysis

## 3.1 Summary of Cryptocurrency Pairs Distribution Across Fiat Currencies

In this section, we explore the distribution of fiat currencies and their pairing with major cryptocurrencies like BTC and USDT. By examining the fiat currencies present in the dataset, particularly focusing on the last month's data, we can identify which fiat currencies are commonly paired with these cryptocurrencies. Additionally, we count the number of unique cryptocurrencies associated with each fiat currency to understand the diversity of crypto-fiat pairs across different fiat currencies.

In [10]:
import pandas as pd

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month]

# Get the list of all fiat currencies in the dataset
all_fiat_currencies = final_df['fiat_currency'].unique()

# Get the list of fiat currencies present in the last month's data
fiat_currencies_last_month = filtered_df['fiat_currency'].unique()

# Identify fiat currencies that have BTC pairs
btc_fiat_currencies = filtered_df[filtered_df['crypto_currency'] == 'BTC']['fiat_currency'].unique()

# Identify fiat currencies that have USDT pairs
usdt_fiat_currencies = filtered_df[filtered_df['crypto_currency'] == 'USDT']['fiat_currency'].unique()

# Identify fiat currencies without BTC pairs
fiat_without_btc = set(fiat_currencies_last_month) - set(btc_fiat_currencies)

# Identify fiat currencies without USDT pairs
fiat_without_usdt = set(fiat_currencies_last_month) - set(usdt_fiat_currencies)

# Identify fiat currencies without either BTC or USDT pairs
fiat_without_btc_or_usdt = fiat_without_btc.union(fiat_without_usdt)

# Counting the number of unique cryptos per fiat currency
crypto_count_per_fiat = final_df.groupby('fiat_currency')['crypto_currency'].nunique()

# Combine all this information into a summary table
summary_table = pd.DataFrame({
    "All Fiat Currencies": [sorted(all_fiat_currencies)],
    "Fiat Currencies in Last Month": [sorted(fiat_currencies_last_month)],
    "Fiat Currencies with BTC Pairs": [sorted(btc_fiat_currencies)],
    "Fiat Currencies with USDT Pairs": [sorted(usdt_fiat_currencies)],
    "Fiat Currencies without BTC Pairs": [sorted(fiat_without_btc)],
    "Fiat Currencies without USDT Pairs": [sorted(fiat_without_usdt)],
    "Fiat Currencies without BTC or USDT Pairs": [sorted(fiat_without_btc_or_usdt)]
})

# Display the summary table as text to make it more readable
for column in summary_table.columns:
    print(f"\n{column} (Total {len(summary_table[column][0])}):")
    print(", ".join(summary_table[column][0]))

# Add a sub-sub-title for the next section
print("\n### Number of Unique Cryptocurrencies per Fiat Currency ###")

# Display the count of unique cryptos per fiat currency in a more readable format
for fiat, count in crypto_count_per_fiat.items():  # Corrected to use items() instead of iteritems()
    print(f"{fiat}: {count} unique cryptocurrencies")



All Fiat Currencies (Total 15):
ARS, AUD, BRL, CZK, EUR, GBP, JPY, MXN, NGN, PLN, RON, RUB, TRY, UAH, ZAR

Fiat Currencies in Last Month (Total 11):
ARS, BRL, CZK, EUR, JPY, MXN, PLN, RON, TRY, UAH, ZAR

Fiat Currencies with BTC Pairs (Total 10):
ARS, BRL, EUR, JPY, MXN, PLN, RON, TRY, UAH, ZAR

Fiat Currencies with USDT Pairs (Total 10):
ARS, BRL, CZK, EUR, MXN, PLN, RON, TRY, UAH, ZAR

Fiat Currencies without BTC Pairs (Total 1):
CZK

Fiat Currencies without USDT Pairs (Total 1):
JPY

Fiat Currencies without BTC or USDT Pairs (Total 2):
CZK, JPY

### Number of Unique Cryptocurrencies per Fiat Currency ###
ARS: 3 unique cryptocurrencies
AUD: 30 unique cryptocurrencies
BRL: 49 unique cryptocurrencies
CZK: 1 unique cryptocurrencies
EUR: 65 unique cryptocurrencies
GBP: 25 unique cryptocurrencies
JPY: 10 unique cryptocurrencies
MXN: 3 unique cryptocurrencies
NGN: 10 unique cryptocurrencies
PLN: 4 unique cryptocurrencies
RON: 4 unique cryptocurrencies
RUB: 25 unique cryptocurrencies
TRY: 

In [11]:
# Diccionario de colores basados en las camisetas de fútbol de cada país (dos colores por país)
country_colors = {
    'USD': ['#B22234', '#3C3B6E'],  # Estados Unidos (rojo y azul)
    'EUR': ['#003399', '#FFD700'],  # Unión Europea (azul y amarillo - colores de la bandera)
    'GBP': ['#FFFFFF', '#CC0000'],  # Reino Unido (blanco y rojo)
    'JPY': ['#BC002D', '#FFFFFF'],  # Japón (rojo y blanco)
    'AUD': ['#FFCC00', '#00843D'],  # Australia (amarillo y verde)
    'CAD': ['#FF0000', '#FFFFFF'],  # Canadá (rojo y blanco)
    'CHF': ['#FF0000', '#FFFFFF'],  # Suiza (rojo y blanco)
    'CNY': ['#FFDE00', '#DE2910'],  # China (amarillo y rojo)
    'INR': ['#FF9933', '#138808'],  # India (naranja y verde)
    'RUB': ['#D52B1E', '#0033A0'],  # Rusia (rojo y azul)
    'BRL': ['#F7E03C', '#00A859'],  # Brasil (amarillo y verde)
    'ARS': ['#74ACDF', '#FFFFFF'],  # Argentina (celeste y blanco)
    'NGN': ['#008751', '#FFFFFF'],  # Nigeria (verde y blanco)
    'TRY': ['#E30A17', '#FFFFFF'],  # Turquía (rojo y blanco)
    'MXN': ['#006847', '#FFFFFF'],  # México (verde y blanco)
    'ZAR': ['#006600', '#FFCC00'],  # Sudáfrica (verde y amarillo)
    'PLN': ['#DC143C', '#FFFFFF'],  # Polonia (rojo y blanco)
    'SEK': ['#FFCD00', '#006AA7'],  # Suecia (amarillo y azul)
    'NOK': ['#BA0C2F', '#00205B'],  # Noruega (rojo y azul)
    'DKK': ['#C60C30', '#FFFFFF'],  # Dinamarca (rojo y blanco)
    'HKD': ['#EE1C25', '#FFFFFF'],  # Hong Kong (rojo y blanco)
    'SGD': ['#E00027', '#FFFFFF'],  # Singapur (rojo y blanco)
    'NZD': ['#000000', '#FFFFFF'],  # Nueva Zelanda (negro y blanco)
    'KRW': ['#C60C30', '#003478'],  # Corea del Sur (rojo y azul)
    'THB': ['#2D2A4A', '#A62712'],  # Tailandia (azul y rojo)
    'MYR': ['#010066', '#FFCC00'],  # Malasia (azul y amarillo)
    'IDR': ['#FF0000', '#FFFFFF'],  # Indonesia (rojo y blanco)
    'PHP': ['#0038A8', '#FFFFFF'],  # Filipinas (azul y blanco)
    'VND': ['#DA251D', '#FFFF00'],  # Vietnam (rojo y amarillo)
    'CZK': ['#DA291C', '#003DA5'],  # República Checa (rojo y azul)
    'HUF': ['#C8102E', '#FFFFFF'],  # Hungría (rojo y blanco)
    'RON': ['#FFD700', '#002D62'],  # Rumanía (amarillo y azul)
    'BGN': ['#00966E', '#FFFFFF'],  # Bulgaria (verde y blanco)
    'HRK': ['#FF0000', '#FFFFFF'],  # Croacia (rojo y blanco)
    'RSD': ['#C6363C', '#FFFFFF'],  # Serbia (rojo y blanco)
    'UAH': ['#FFD700', '#0057B7'],  # Ucrania (amarillo y azul)
    'KZT': ['#00AFCA', '#FFD700'],  # Kazajistán (azul y amarillo)
    'AZN': ['#00B140', '#EF3340'],  # Azerbaiyán (verde y rojo)
    'GEL': ['#DD0034', '#FFFFFF'],  # Georgia (rojo y blanco)
    'BYN': ['#D22630', '#FFFFFF'],  # Bielorrusia (rojo y blanco)
    'EGP': ['#CE1126', '#FFFFFF'],  # Egipto (rojo y blanco)
    'ILS': ['#0038A8', '#FFFFFF'],  # Israel (azul y blanco)
    'SAR': ['#00843D', '#FFFFFF'],  # Arabia Saudita (verde y blanco)
    'AED': ['#000000', '#FFFFFF'],  # Emiratos Árabes Unidos (negro y blanco)
    'QAR': ['#8C1D40', '#FFFFFF'],  # Qatar (granate y blanco)
    'KWD': ['#007A3D', '#FFFFFF'],  # Kuwait (verde y blanco)
    'BHD': ['#8C1D40', '#FFFFFF'],  # Baréin (granate y blanco)
    'OMR': ['#D71920', '#FFFFFF'],  # Omán (rojo y blanco)
    'TND': ['#DA291C', '#FFFFFF'],  # Túnez (rojo y blanco)
    'MAD': ['#C1272D', '#4CBB17'],  # Marruecos (rojo y verde)
    'DZD': ['#007A33', '#FFFFFF'],  # Argelia (verde y blanco)
    'TWD': ['#0033A0', '#FFFFFF'],  # Taiwán (azul y blanco)
    'LKR': ['#FEC334', '#B22222'],  # Sri Lanka (amarillo y rojo)
    'PKR': ['#01411C', '#FFFFFF'],  # Pakistán (verde y blanco)
    'BDT': ['#006A4E', '#FFFFFF'],  # Bangladés (verde y blanco)
    'KES': ['#006600', '#FFFFFF'],  # Kenia (verde y blanco)
    'TZS': ['#008C45', '#FFFFFF'],  # Tanzania (verde y blanco)
    'UGX': ['#FFD100', '#CE1126'],  # Uganda (amarillo y rojo)
    'GHS': ['#006B3F', '#FFCC00'],  # Ghana (verde y amarillo)
    'XAF': ['#FFD700', '#00853F'],  # África Central (amarillo y verde)
    'XOF': ['#FFD700', '#00853F'],  # África Occidental (amarillo y verde)
    'XPF': ['#00467F', '#FFFFFF'],  # Polinesia Francesa (azul y blanco)
    'MUR': ['#EC1C24', '#FFD100'],  # Mauricio (rojo y amarillo)
    'SCR': ['#003DA5', '#FFFFFF'],  # Seychelles (azul y blanco)
    'MGA': ['#E20D18', '#007A33'],  # Madagascar (rojo y verde)
    'ZMW': ['#FF9933', '#008751'],  # Zambia (naranja y verde)
    'MWK': ['#CE1126', '#000000'],  # Malaui (rojo y negro)
    'BWP': ['#00ADEF', '#FFFFFF'],  # Botsuana (azul claro y blanco)
    'NAD': ['#00247D', '#FFD100'],  # Namibia (azul y amarillo)
    'SLL': ['#0072C6', '#FFFFFF'],  # Sierra Leona (azul y blanco)
    'LRD': ['#B22234', '#3C3B6E'],  # Liberia (rojo y azul)
    'LSL': ['#003DA5', '#FFFFFF'],  # Lesoto (azul y blanco)
    'SZL': ['#00247D', '#FFD100'],  # Esuatini (azul y amarillo)
    'MZN': ['#C8102E', '#000000'],  # Mozambique (rojo y negro)
    'BIF': ['#00853F', '#FFFFFF'],  # Burundi (verde y blanco)
    'CDF': ['#007FFF', '#FFD700'],  # Congo (azul y amarillo)
    'DJF': ['#00ADEF', '#FFFFFF'],  # Yibuti (azul claro y blanco)
    'ETB': ['#DA291C', '#007A33'],  # Etiopía (rojo y verde)
    'GNF': ['#FFD100', '#CE1126'],  # Guinea (amarillo y rojo)
    'HTG': ['#00205B', '#CE1126'],  # Haití (azul y rojo)
    'NPR': ['#DC143C', '#003153'],  # Nepal (rojo y azul)
    'PGK': ['#D71920', '#000000'],  # Papúa Nueva Guinea (rojo y negro)
    'SHP': ['#00205B', '#FFFFFF'],  # Santa Helena (azul y blanco)
    'SOM': ['#418FDE', '#FFFFFF'],  # Somalia (azul claro y blanco)
    'SSP': ['#CE1126', '#000000'],  # Sudán del Sur (rojo y negro)
    'SYP': ['#CE1126', '#FFFFFF'],  # Siria (rojo y blanco)
    'TTD': ['#E0001B', '#FFFFFF'],  # Trinidad y Tobago (rojo y blanco)
    'TOP': ['#C8102E', '#FFFFFF'],  # Tonga (rojo y blanco)
    'TWD': ['#0033A0', '#FFFFFF'],  # Taiwán (azul y blanco)
    'VUV': ['#FFD100', '#008751'],  # Vanuatu (amarillo y verde)
    'WST': ['#003DA5', '#FFFFFF'],  # Samoa (azul y blanco)
    'XCD': ['#00205B', '#CE1126'],  # Caribe Oriental (azul y rojo)
    'YER': ['#CE1126', '#FFFFFF'],  # Yemen (rojo y blanco)
}

# Asegúrate de que todos los países presentes en el gráfico tengan colores asignados
for country in fiat_currencies:
    if country not in country_colors:
        country_colors[country] = ['#CCCCCC', '#999999']  # Asignar colores grises por defecto si no está definido


In [12]:
import plotly.graph_objects as go

# Ordenar los datos de mayor a menor
crypto_count_per_fiat_sorted = crypto_count_per_fiat.sort_values(ascending=False)

# Crear un gráfico de barras para mostrar el número de criptomonedas únicas por moneda fiat
fig_crypto_count_per_fiat = go.Figure()

for country in crypto_count_per_fiat_sorted.index:
    # Obtener el color asociado al país (usando el primer color de la lista)
    color = country_colors.get(country, ['#CCCCCC', '#999999'])[0]  # Usar gris si no se encuentra el país
    
    fig_crypto_count_per_fiat.add_trace(go.Bar(
        x=[country],  # Agregar el país como única entrada en el eje X
        y=[crypto_count_per_fiat_sorted[country]],  # Agregar el valor correspondiente
        marker_color=color,  # Asignar el color de la camiseta de fútbol
        name=country
    ))

# Configurar el layout del gráfico
fig_crypto_count_per_fiat.update_layout(
    title='<b>Number of Unique Cryptocurrencies per Fiat Currency<b>',
    title_font_color="#AA322F",  
    xaxis_title='Fiat Currency',
    yaxis_title='Number of Unique Cryptocurrencies',
    xaxis_tickangle=-45,
    showlegend=False  # Ocultar la leyenda si no es necesaria
)

# Mostrar el gráfico
fig_crypto_count_per_fiat.show()


In [13]:
fig_btc_usdt_pairs = go.Figure()

# Agregar las barras para BTC
fig_btc_usdt_pairs.add_trace(go.Bar(
    x=list(fiat_currencies_last_month),
    y=[1 if fiat in btc_fiat_currencies else 0 for fiat in fiat_currencies_last_month],
    name='BTC Pairs',
    marker_color='dodgerblue'
))

# Agregar las barras para USDT
fig_btc_usdt_pairs.add_trace(go.Bar(
    x=list(fiat_currencies_last_month),
    y=[1 if fiat in usdt_fiat_currencies else 0 for fiat in fiat_currencies_last_month],
    name='USDT Pairs',
    marker_color='orange'
))

# Configurar el layout del gráfico
fig_btc_usdt_pairs.update_layout(
    barmode='stack',
    title='<b>Fiat Currencies with BTC and USDT Pairs<b>',
    title_font_color="#AA322F",   
    xaxis_title='Fiat Currency',
    yaxis_title='Presence (1 = Yes, 0 = No)',
    xaxis_tickangle=-45,
)

# Mostrar el gráfico
fig_btc_usdt_pairs.show()

In [14]:
import pandas as pd
import plotly.graph_objects as go

# Asegurarse de que 'timestamp' esté en formato datetime
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Filtrar los datos para incluir solo los datos del último mes
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month]

# Eliminar duplicados para contar cada par cripto-fiat solo una vez
unique_pairs_df = filtered_df.drop_duplicates(subset=['crypto_currency', 'fiat_currency'])

# Contar la frecuencia de cada criptomoneda en el conjunto de datos filtrados
crypto_frequency = unique_pairs_df['crypto_currency'].value_counts()

# Seleccionar las 15 criptomonedas principales
top_15_cryptos = crypto_frequency.nlargest(15)

# Diccionario de colores para criptomonedas (puedes ajustar los colores según prefieras)
crypto_colors = {
    'BTC': '#F7931A',  # Bitcoin
    'ETH': '#3C3C3D',  # Ethereum
    'USDT': '#26A17B',  # Tether
    'BNB': '#F3BA2F',  # Binance Coin
    'ADA': '#0033AD',  # Cardano
    'XRP': '#23292F',  # Ripple
    'SOL': '#00FFA3',  # Solana
    'DOT': '#E6007A',  # Polkadot
    'DOGE': '#C2A633',  # Dogecoin
    'LTC': '#BEBEBE',  # Litecoin
    'LINK': '#2A5ADA',  # Chainlink
    'BCH': '#8DC351',  # Bitcoin Cash
    'XLM': '#14B8EC',  # Stellar
    'ATOM': '#2E3148',  # Cosmos
    'VET': '#15BDFF',  # VeChain
}

# Crear el gráfico de barras de Plotly con un nombre descriptivo
fig_unique_frequency = go.Figure()

for crypto in top_15_cryptos.index:
    color = crypto_colors.get(crypto, 'skyblue')  # Asignar color de criptomoneda o skyblue por defecto
    fig_unique_frequency.add_trace(go.Bar(
        x=[crypto],
        y=[top_15_cryptos[crypto]],
        marker_color=color,
        name=crypto
    ))

# Personalizar el diseño del gráfico
fig_unique_frequency.update_layout(
    title="<b>Unique Frequency of Top 15 Cryptocurrencies in the Last Month<b>",
    title_font_color="#AA322F",   
    xaxis_title="Cryptocurrency",
    yaxis_title="Unique Frequency",
    xaxis_tickangle=-45,
    template='plotly_white',
    showlegend=False  # Ocultar la leyenda si no es necesaria
)

# Mostrar el gráfico
fig_unique_frequency.show()

# Mensaje de confirmación simple
print("Unique frequency plot of top 15 cryptocurrencies for the last month generated.")


Unique frequency plot of top 15 cryptocurrencies for the last month generated.


In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Ordenar los datos de mayor a menor para el primer gráfico
crypto_count_per_fiat_sorted = crypto_count_per_fiat.sort_values(ascending=False)

# Crear la figura con 1 fila y 2 columnas
fig = make_subplots(rows=1, cols=2, subplot_titles=("Unique Cryptocurrencies per Fiat", 
                                                   "Frequency of Top 15 Cryptocurrencies"))

# Gráfico 1: Número de criptomonedas únicas por moneda fiat (ordenado)
for country in crypto_count_per_fiat_sorted.index:
    color = country_colors.get(country, ['#CCCCCC', '#999999'])[0]
    fig.add_trace(go.Bar(
        x=[country],  
        y=[crypto_count_per_fiat_sorted[country]],  
        marker_color=color,  
        name=country
    ), row=1, col=1)

# Gráfico 2: Frecuencia única de las 15 principales criptomonedas en el último mes
for crypto in top_15_cryptos.index:
    color = crypto_colors.get(crypto, 'skyblue')
    fig.add_trace(go.Bar(
        x=[crypto],
        y=[top_15_cryptos[crypto]],
        marker_color=color,
        name=crypto
    ), row=1, col=2)

# Ajustar el diseño final
fig.update_layout(
    height=600, 
    width=1000, 
    title_text="Comparative Analysis of Cryptoasset Metrics",
    title_font=dict(size=20, color="#AA322F"),
    showlegend=False,
    title_x=0.5  
)

# Mostrar la figura
fig.show()


In [16]:
## Heatmap of Top 10 Cryptocurrencies Paired with Fiat Currencies

# This section presents a heatmap that visualizes the presence of trading pairs between the top 10 cryptocurrencies and various fiat currencies. The heatmap offers a clear overview of which fiat currencies are most commonly paired with these leading cryptocurrencies. Additionally, we include a category labeled 'Others' to capture any cryptocurrencies outside the top 10, ensuring a comprehensive view of the dataset.

import pandas as pd
import plotly.graph_objects as go

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Count the number of pairs for each cryptocurrency across the entire dataset
crypto_counts = final_df['crypto_currency'].value_counts()

# Identify the top 10 cryptocurrencies based on the entire dataset
top_10_cryptos = crypto_counts.nlargest(10).index.tolist()

# Filter the data to include only the last month's data
last_month = final_df['timestamp'].max() - pd.DateOffset(months=1)
filtered_df = final_df[final_df['timestamp'] >= last_month].copy()  # Use .copy() to avoid the SettingWithCopyWarning

# Group the remaining cryptocurrencies into "Others"
filtered_df.loc[:, 'crypto_category'] = filtered_df['crypto_currency'].apply(
    lambda x: x if x in top_10_cryptos else 'Others'
)

# Create a list of unique categories (top 10 cryptos + 'Others')
categories = top_10_cryptos + ['Others']

# List of all fiat currencies present in the entire dataset
all_fiats = sorted(final_df['fiat_currency'].unique())  # Sort fiat currencies alphabetically

# Initialize an empty matrix with zeros (no pairs) for all fiat currencies
matrix = pd.DataFrame(0, index=categories, columns=all_fiats)

# Fill the matrix with 1 where pairs exist
for _, row in filtered_df.iterrows():
    matrix.loc[row['crypto_category'], row['fiat_currency']] = 1

# Unified color scheme for all cryptos
colorscale = [[0, '#D3D3D3'], [1, '#4682B4']]  # Light gray for no pair, steel blue for all pairs

# Plot using Plotly
fig_heatmap_top10 = go.Figure(data=go.Heatmap(
    z=matrix.values,
    x=matrix.columns,
    y=matrix.index,
    colorscale=colorscale,
    showscale=False,  # Hide color scale bar
))

# Update layout for better readability and custom Y-axis labels
fig_heatmap_top10.update_layout(
    title="<b>Top 10 Crypto-Fiat Pair Presence and 'Others'<b>",
    title_font_color="#AA322F",   
    xaxis_title="Fiat Currency",
    yaxis_title="Cryptocurrency",
    yaxis=dict(
        tickmode='array',
        tickvals=list(range(len(categories))),
        ticktext=[f'<b>{crypto}</b>' if crypto == 'Others' else crypto for crypto in categories],
        tickfont=dict(size=10),
        side='left'
    ),
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(len(all_fiats))),
        ticktext=all_fiats  # Ensure all fiat currencies are shown on the X-axis
    ),
    template="plotly_white"
)

# Show the plot
fig_heatmap_top10.show()


In [17]:
# Animated Heatmap of Crypto-Fiat Pairs Over Time

# In this section, we extend the heatmap analysis by introducing a time element, creating an animated heatmap that shows the presence of crypto-fiat pairs over a series of months. This visualization helps track changes over time, providing insights into the evolving dynamics of crypto-fiat pairings in the market. The animation allows us to see how the presence of different pairs fluctuates across different time periods, offering a temporal dimension to our analysis.

import pandas as pd
import plotly.graph_objects as go
import warnings
from tqdm.notebook import tqdm  # To show progress bar

# Suppress the SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Ensure 'timestamp' is in datetime format
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

# Count the number of pairs for each cryptocurrency across the entire dataset
all_crypto_counts = final_df['crypto_currency'].value_counts()

# Identify the top 10 cryptocurrencies based on the entire dataset
top_10_cryptos = all_crypto_counts.nlargest(10).index.tolist()

# Define categories for Y-axis (top 10 cryptos + 'Others')
categories = top_10_cryptos + ['Others']

# Extract the month from the timestamp
final_df['month'] = final_df['timestamp'].dt.to_period('M')

# Extend the data to include up to 2024-08, even if there are no new data entries
all_months = pd.period_range(final_df['month'].min(), '2024-08', freq='M')

# List of all fiat currencies present in the dataset
all_fiats = sorted(final_df['fiat_currency'].unique())  # Sort fiat currencies alphabetically

# Initialize a list to store all frames
frames = []

# Generate a frame for each month
for month in tqdm(all_months, desc="Generating frames"):
    # Filter the data for the given month
    monthly_data = final_df[final_df['month'] == month].copy()

    # Group the remaining cryptocurrencies into "Others"
    if not monthly_data.empty:
        monthly_data.loc[:, 'crypto_category'] = monthly_data['crypto_currency'].apply(
            lambda x: x if x in top_10_cryptos else 'Others'
        )

    # Initialize an empty matrix with zeros (no pairs)
    matrix = pd.DataFrame(0, index=categories, columns=all_fiats)

    # Fill the matrix with 1 where pairs exist
    if not monthly_data.empty:
        for _, row in monthly_data.iterrows():
            matrix.loc[row['crypto_category'], row['fiat_currency']] = 1

    # Create a frame for this month
    frames.append(go.Frame(
        data=go.Heatmap(
            z=matrix.values,
            x=matrix.columns,
            y=matrix.index,
            colorscale=[[0, '#D3D3D3'], [1, '#4682B4']],  # Light gray for no pair, steel blue for all pairs
            showscale=False
        ),
        name=str(month)
    ))

# Plot using Plotly
fig_heatmap_animated = go.Figure(
    data=frames[0]['data'],  # Use the first month's data as the starting point
    layout=go.Layout(
        title="<b>Top 10 Crypto-Fiat Pair Presence and 'Others' Over Time (All Fiat Currencies)<b>",
        title_font_color="#AA322F",   
        xaxis_title="Fiat Currency",
        yaxis_title="Cryptocurrency",
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(categories))),
            ticktext=[f'<b>{crypto}</b>' if crypto == 'Others' else crypto for crypto in categories],
            tickfont=dict(size=10),
            side='left'
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(len(all_fiats))),
            ticktext=all_fiats  # This is now sorted alphabetically
        ),
        template="plotly_white",
        updatemenus=[dict(type="buttons", showactive=False,
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 500, "redraw": True},
                                                     "fromcurrent": True,
                                                     "transition": {"duration": 300}}]),
                                   dict(label="Pause",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                       "mode": "immediate",
                                                       "transition": {"duration": 0}}])])],
        sliders=[{
            'currentvalue': {
                'prefix': 'Month: ',
                'font': {'size': 20}
            },
            'pad': {'b': 10},
            'len': 0.9,
            'x': 0.1,
            'y': 0,
            'steps': [{
                'args': [[str(month)], {
                    'frame': {'duration': 500, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 300}
                }],
                'label': str(month),
                'method': 'animate'
            } for month in all_months]
        }]
    ),
    frames=frames  # Add the frames to the figure
)

# Show the plot
fig_heatmap_animated.show()


Generating frames:   0%|          | 0/59 [00:00<?, ?it/s]

## 3.2 USD Volume

Block 1: Download prices in USD

In [18]:
pip install coinmetrics-api-client -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [19]:
from coinmetrics.api_client import CoinMetricsClient
import pandas as pd
from tqdm.notebook import tqdm

# Inicializa el cliente de Coin Metrics para la API Comunitaria
client = CoinMetricsClient()

# Define los activos y métricas
assets = ['btc', 'eth', 'usdt']
metrics = ['PriceUSD']

# Define la fecha de inicio (January 1, 2023)
start_time = '2019-01-01'

# Prepara la barra de progreso y descarga los precios
price_data_frames = []
for asset in tqdm(assets, desc="Downloading prices"):
    # Descarga los datos de precios para cada activo
    price_data = client.get_asset_metrics(assets=[asset], metrics=metrics, frequency='1d', start_time=start_time)
    
    # Convierte los datos a un DataFrame de Pandas y agrégalo a la lista
    price_data_frames.append(price_data.to_dataframe())

# Combina todos los DataFrames en uno solo
price_df = pd.concat(price_data_frames)

# Asegúrate de que la columna 'PriceUSD' sea numérica
price_df['PriceUSD'] = pd.to_numeric(price_df['PriceUSD'], errors='coerce')

Block 2: Processing data and calculating volumeUSD

In [20]:
# Normalizar zonas horarias
# Convertir las columnas de tiempo a datetime sin zona horaria
filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp']).dt.tz_localize(None)
price_df['time'] = pd.to_datetime(price_df['time']).dt.tz_localize(None)

# Filtrar el DataFrame para incluir solo BTC, ETH, y USDT
filtered_df = final_df[final_df['crypto_currency'].isin(['BTC', 'ETH', 'USDT'])]

# Convertir ambos a minúsculas para asegurar coincidencia
filtered_df['crypto_currency'] = filtered_df['crypto_currency'].str.lower()
price_df['asset'] = price_df['asset'].str.lower()

# Fusionar los DataFrames en función de 'timestamp', 'crypto_currency', y 'asset'
merged_df = pd.merge(filtered_df, price_df, left_on=['timestamp', 'crypto_currency'], right_on=['time', 'asset'], how='inner')

# Crear la columna 'volumeUSD' realizando la multiplicación del volumen por el precio en USD
merged_df['volumeUSD'] = merged_df['volume'] * merged_df['PriceUSD']

# Crear una nueva base de datos con las columnas relevantes
new_base = merged_df[['timestamp', 'crypto_currency', 'volumeUSD', 'fiat_currency']]

# Renombrar las columnas para mayor claridad (opcional)
new_base.columns = ['Date', 'Asset', 'VolumeUSD', 'Country']

In [21]:
import igraph as ig
import plotly.graph_objects as go

# Supongamos que 'new_base' ya está cargado en el entorno, aquí replicamos lo necesario para el gráfico

# Extraer los vertices y las aristas a partir de 'new_base'
vertices = set()
edges = []
weights = []

for _, row in new_base.iterrows():
    asset = row['Asset']
    country = row['Country']
    volume = row['VolumeUSD']
    
    vertices.add(asset)
    vertices.add(country)
    edges.append((asset, country))
    weights.append(volume)

# Convertir el conjunto de vertices a una lista
vertices = list(vertices)

# Crear el grafo en igraph
G_ig = ig.Graph()
G_ig.add_vertices(vertices)
G_ig.add_edges(edges)
G_ig.es['weight'] = weights  # Asignar los pesos de las aristas

# Filtrar nodos con grado menor que 3
low_degree_nodes = [v.index for v in G_ig.vs if G_ig.degree(v) < 1]
G_ig.delete_vertices(low_degree_nodes)

# Calcular PageRank para los nodos restantes y aplicar un escalado
pagerank = G_ig.pagerank(weights=G_ig.es['weight'])
max_pagerank = max(pagerank)

# Exagerar las diferencias aplicando una transformación exponencial al PageRank
min_size = 1  # Tamaño mínimo para nodos con solo 3 conexiones
mid_size = 10 # Tamaño para nodos con 4 conexiones
max_size = 40 # Tamaño máximo para nodos con más conexiones y alto PageRank

# Asignar tamaños basados en el número de conexiones y PageRank
vertex_sizes = []
for i in range(len(G_ig.vs)):
    degree = G_ig.degree(i)
    if degree == 3:
        vertex_sizes.append(min_size)
    elif degree == 4:
        vertex_sizes.append(mid_size)
    else:
        vertex_sizes.append(min_size + (pagerank[i] / max_pagerank) ** 0.3 * (max_size - min_size))

# Asignar colores y etiquetas a los nodos
vertex_colors = []
label_colors = []
for i in range(len(G_ig.vs)):
    if G_ig.vs[i]['name'] in new_base['Country'].unique():
        vertex_colors.append('blue')
        label_colors.append('blue')
    else:
        vertex_colors.append('red')
        label_colors.append('red')

# Definir tamaños de las etiquetas basados en PageRank
vertex_labels = [G_ig.vs[i]['name'] for i in range(len(G_ig.vs))]

# Plotear el grafo utilizando Plotly para visualización interactiva
layout = G_ig.layout('fr')  # Usar el layout Fruchterman-Reingold
Xn = [layout[k][0] for k in range(len(layout))]
Yn = [layout[k][1] for k in range(len(layout))]
Xe = []
Ye = []
for e in G_ig.es:
    Xe.extend([layout[e.source][0], layout[e.target][0], None])
    Ye.extend([layout[e.source][1], layout[e.target][1], None])

fig = go.Figure()

# Agregar aristas
fig.add_trace(go.Scatter(x=Xe, y=Ye,
                         mode='lines',
                         line=dict(width=1, color='grey'),
                         hoverinfo='none'
                         ))

# Agregar nodos
fig.add_trace(go.Scatter(x=Xn, y=Yn,
                         mode='markers+text',
                         marker=dict(symbol='circle-dot',
                                     size=vertex_sizes,
                                     color=vertex_colors,
                                     line=dict(color='black', width=0.5)),
                         text=vertex_labels,
                         textposition='top center',
                         textfont=dict(size=10),
                         hoverinfo='text'
                         ))

fig.update_layout(showlegend=False)
fig.show()



In [22]:
new_base['Year'] = pd.to_datetime(new_base['Date']).dt.year

In [23]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import igraph as ig

# Definir el diccionario para acumular volúmenes
total_volume_dict = {}

# Calcular los volúmenes totales por nodo
for _, row in new_base.iterrows():
    asset = row['Asset']
    country = row['Country']
    volume = row['VolumeUSD']
    
    # Acumular volúmenes
    total_volume_dict[asset] = total_volume_dict.get(asset, 0) + volume
    total_volume_dict[country] = total_volume_dict.get(country, 0) + volume

# Determinar el volumen máximo
max_volume = max(total_volume_dict.values())

# Crear una figura con 4 subplots (2x2)
fig = make_subplots(rows=2, cols=2, subplot_titles=("2021", "2022", "2023", "2024"),
                    vertical_spacing=0.1)  # Reducir espacio entre filas

# Definir los años para cada subplot
years = [2021, 2022, 2023, 2024]
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for year, pos in zip(years, positions):
    yearly_data = new_base[new_base['Year'] == year]  # Filtrar los datos por año
    vertices = set()
    edges = []
    weights = []
    
    for _, row in yearly_data.iterrows():
        asset = row['Asset']
        country = row['Country']
        volume = row['VolumeUSD']
        
        vertices.add(asset)
        vertices.add(country)
        edges.append((asset, country))
        weights.append(volume)
    
    vertices = list(vertices)
    G_ig = ig.Graph()
    G_ig.add_vertices(vertices)
    G_ig.add_edges(edges)
    G_ig.es['weight'] = weights
    
    layout = G_ig.layout('circle')
    Xn = [layout[k][0] for k in range(len(layout))]
    Yn = [layout[k][1] for k in range(len(layout))]
    Xe = []
    Ye = []
    for e in G_ig.es:
        Xe.extend([layout[e.source][0], layout[e.target][0], None])
        Ye.extend([layout[e.source][1], layout[e.target][1], None])
    
    vertex_sizes = [(total_volume_dict.get(v, 0) / max_volume) * 40 + 10 for v in G_ig.vs['name']]
    vertex_colors = [aes_color if v in aes_nodes else emdes_color if v in emdes_nodes else '#D3D3D3' for v in G_ig.vs['name']]  # Criptomonedas en gris
    
    # Agregar nodos al subplot correspondiente
    fig.add_trace(go.Scatter(x=Xn, y=Yn,
                             mode='markers+text',
                             marker=dict(symbol='circle',
                                         size=vertex_sizes,
                                         color=vertex_colors,
                                         line=dict(color='black', width=0.5)),
                             text=G_ig.vs['name'],
                             textposition='top center',
                             textfont=dict(size=10),
                             hoverinfo='text'),
                  row=pos[0], col=pos[1])
    
    # Agregar aristas con grosor específico para cada traza
    for i, e in enumerate(G_ig.es):
        fig.add_trace(go.Scatter(x=[layout[e.source][0], layout[e.target][0], None],
                                 y=[layout[e.source][1], layout[e.target][1], None],
                                 mode='lines',
                                 line=dict(width=(weights[i] / max(weights)) * 5, color='grey'),  # Usar grosor basado en `weights`
                                 hoverinfo='none'),
                      row=pos[0], col=pos[1])

# Ajustar el diseño de la figura
fig.update_layout(height=800, width=800, showlegend=False,
                  title_text="<b>Annual Evolution of the Crypto and Fiat Network<b>",
                  title_font=dict(size=20, color="#AA322F"),
                  xaxis=dict(showticklabels=False, zeroline=False),  # Quitar escalas de los ejes
                  yaxis=dict(showticklabels=False, zeroline=False))  # Quitar escalas de los ejes

# Aplicar eliminación de escalas a todos los subplots
for i in range(1, 5):
    fig.update_xaxes(showticklabels=False, row=(i+1)//2, col=(i-1)%2 + 1)
    fig.update_yaxes(showticklabels=False, row=(i+1)//2, col=(i-1)%2 + 1)

fig.show()


NameError: name 'aes_nodes' is not defined

In [ ]:
import igraph as ig
import plotly.graph_objects as go
import pandas as pd

# Asegúrate de que la columna de meses exista
if 'Date' in new_base.columns:
    new_base['Month'] = pd.to_datetime(new_base['Date']).dt.to_period('M')

# Definir los colores
aes_color = '#AA322F'  # Rojo oscuro para AEs
emdes_color = '#6FA5E2'  # Azul pastel para EMDEs
crypto_color = '#D3D3D3'  # Gris claro para criptomonedas
inactive_color = '#D3D3D3'  # Gris claro tenue para nodos inactivos

x_range = [-1.5, 1.5]
y_range = [-1.5, 1.5]

# Listas de nodos para AEs y EMDEs
aes_nodes = sorted(set(new_base[new_base['Country'].isin(aes_fiat_currencies)]['Country'].unique()))
emdes_nodes = sorted(set(new_base[new_base['Country'].isin(emdes_fiat_currencies)]['Country'].unique()))
crypto_nodes = sorted(set(new_base['Asset'].unique()))

# Definir la lista de meses que quieres analizar
months = sorted(new_base['Month'].unique())

# Estados iniciales de los nodos (todos apagados)
node_states = {node: inactive_color for node in aes_nodes + emdes_nodes + crypto_nodes}

frames = []
previous_nodes = set()

for month in months:
    monthly_data = new_base[new_base['Month'] == month]
    vertices = set()
    edges = []
    weights = []
    
    for _, row in monthly_data.iterrows():
        asset = row['Asset']
        country = row['Country']
        volume = row['VolumeUSD']
        
        vertices.add(asset)
        vertices.add(country)
        edges.append((asset, country))
        weights.append(volume)
    
    current_nodes = vertices
    added_nodes = current_nodes - previous_nodes
    removed_nodes = previous_nodes - current_nodes
    
    # Actualizar el estado de los nodos
    for node in added_nodes:
        if node in aes_nodes:
            node_states[node] = aes_color  # Encender en color rojo oscuro
        elif node in emdes_nodes:
            node_states[node] = emdes_color  # Encender en color azul pastel
        else:
            node_states[node] = inactive_color  # Encender en gris claro
    
    for node in removed_nodes:
        node_states[node] = inactive_color  # Volver al gris claro tenue si desaparece
    
    vertices = list(vertices)
    G_ig = ig.Graph()
    G_ig.add_vertices(vertices)
    G_ig.add_edges(edges)
    G_ig.es['weight'] = weights
    
    layout = G_ig.layout('circle')
    Xn = [layout[k][0] for k in range(len(layout))]
    Yn = [layout[k][1] for k in range(len(layout))]
    Xe = []
    Ye = []
    for e in G_ig.es:
        Xe.extend([layout[e.source][0], layout[e.target][0], None])
        Ye.extend([layout[e.source][1], layout[e.target][1], None])
    
    vertex_sizes = [(total_volume_dict.get(v, 0) / max_volume) * 40 + 10 for v in G_ig.vs['name']]
    
    vertex_colors = []
    vertex_shapes = []
    for v in G_ig.vs['name']:
        if v in aes_nodes:
            vertex_colors.append(aes_color)
            vertex_shapes.append('triangle-up')
        elif v in emdes_nodes:
            vertex_colors.append(emdes_color)
            vertex_shapes.append('triangle-up')
        else:
            vertex_colors.append(crypto_color)
            vertex_shapes.append('circle')
    
    fig = go.Scatter(x=Xn, y=Yn,
                     mode='markers+text',
                     marker=dict(symbol=vertex_shapes,
                                 size=vertex_sizes,
                                 color=vertex_colors,
                                 line=dict(color='black', width=0.5)),
                     text=G_ig.vs['name'],
                     textposition='top center',
                     textfont=dict(size=10),
                     hoverinfo='text'
                     )

    edge_trace = go.Scatter(x=Xe, y=Ye,
                            mode='lines',
                            line=dict(width=1, color='grey'),
                            hoverinfo='none')

    # Crear las columnas de nodos con estado de color
    aes_annotations = [
        dict(
            xref='paper', yref='paper', x=-0.3, y=1.03 - i * 0.045, showarrow=False,
            text=node,
            font=dict(size=11, color=node_states[node]),
            align="left"
        ) for i, node in enumerate(aes_nodes)
    ]

    emdes_annotations = [
        dict(
            xref='paper', yref='paper', x=-0.15, y=1.03 - i * 0.045, showarrow=False,
            text=node,
            font=dict(size=11, color=node_states[node]),
            align="left"
        ) for i, node in enumerate(emdes_nodes)
    ]

    frame_annotations = aes_annotations + emdes_annotations

    frames.append(go.Frame(data=[fig, edge_trace], name=str(month), layout=go.Layout(annotations=frame_annotations)))

    previous_nodes = current_nodes

layout = go.Layout(
    title="Crypto Trading Network Over Time",
    showlegend=False,
    xaxis=dict(range=x_range, autorange=False),
    yaxis=dict(range=y_range, autorange=False),
    updatemenus=[{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 1000, "redraw": True},
                                "fromcurrent": True, "transition": {"duration": 300}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                  "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "steps": [
            {
                "args": [[str(month)], {"frame": {"duration": 300, "redraw": True},
                                        "mode": "immediate", "transition": {"duration": 300}}],
                "label": str(month),
                "method": "animate"
            } for month in months
        ],
        "currentvalue": {"prefix": "Month: ", "visible": True, "xanchor": "center"},
        "transition": {"duration": 300},
        "x": 0.1,
        "xanchor": "center",
        "y": -0.1,
        "yanchor": "top"
    }]
)

fig = go.Figure(
    data=[frames[0].data[0], frames[0].data[1]],
    layout=layout,
    frames=frames
)

fig.show()


### By country

In [ ]:
# Group by month

# Crear una columna de mes
new_base['Month'] = new_base['Date'].dt.to_period('M')

# Agrupar por mes y país y sumar los volúmenes en USD
grouped_data = new_base.groupby(['Month', 'Country'])['VolumeUSD'].sum().unstack().fillna(0)

In [ ]:
import plotly.graph_objects as go

# Crear un gráfico de barras apiladas utilizando dos colores por país
fig_vol_countries = go.Figure()

for idx, country in enumerate(grouped_data.columns):
    # Alternar entre los dos colores del país para las barras
    color = country_colors[country][idx % 2]
    
    fig_vol_countries.add_trace(go.Bar(
        x=grouped_data.index.astype(str),  # Convertir a cadena para que se muestren correctamente los meses
        y=grouped_data[country],
        name=country,
        marker_color=color  # Asignar el color de la camiseta de fútbol
    ))

# Configurar el layout del gráfico
fig_vol_countries.update_layout(
    barmode='stack',  # Apilar las barras
    title='<b>Cryptoasset Exchange Volumes by Local Currencies by Country<b>',
    title_font_color="#AA322F",    
    xaxis_title='Date',
    yaxis_title='Volume (USD)',
    legend_title_text='Country'
)

# Mostrar el gráfico
fig_vol_countries.show()


### By crypto

In [ ]:
# Crear una columna de mes
new_base['Month'] = new_base['Date'].dt.to_period('M')

# Agrupar por mes y criptomoneda y sumar los volúmenes en USD
grouped_data_crypto = new_base.groupby(['Month', 'Asset'])['VolumeUSD'].sum().unstack().fillna(0)

# Verifica los primeros registros para asegurarte de que los datos están listos para el gráfico
print(grouped_data_crypto.head())


In [ ]:
import plotly.graph_objects as go

# Definir los colores para cada criptomoneda
crypto_colors = {
    'btc': '#F7931A',   # Naranja para Bitcoin
    'eth': '#627EEA',   # Azul para Ethereum
    'usdt': '#26A17B',  # Verde para Tether
}

# Crear el gráfico de barras apiladas
fig_vol_cryptos = go.Figure()

for crypto in grouped_data_crypto.columns:
    fig_vol_cryptos.add_trace(go.Bar(
        x=grouped_data_crypto.index.astype(str),  # Convertir a cadena para que se muestren correctamente los meses
        y=grouped_data_crypto[crypto],
        name=crypto,
        marker_color=crypto_colors[crypto]  # Asignar el color correspondiente a la criptomoneda
    ))

# Configurar el layout del gráfico
fig_vol_cryptos.update_layout(
    title='<b>Cryptoasset Exchange Volumes by Cryptocurrency<b>',
    title_font_color="#AA322F",    
    xaxis_title='Date',
    yaxis_title='Volume (USD)',
    barmode='stack',  # Asegura que las barras estén apiladas
    showlegend=True  # Ocultar la leyenda
)

# Mostrar el gráfico
fig_vol_cryptos.show()

In [ ]:
from plotly.subplots import make_subplots

# Crear la figura con 1 fila y 2 columnas
fig = make_subplots(rows=1, cols=2, subplot_titles=("by Cryptocurrency", 
                                                   "by Country"))

# Agregar las trazas de los gráficos ya existentes a los subplots
for trace in fig_vol_cryptos['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in fig_vol_countries['data']:
    fig.add_trace(trace, row=1, col=2)

# Actualizar layout para cada subplot
fig.update_traces(marker=dict(line=dict(width=0)), selector=dict(type='bar'))

# Ajustar el diseño para la figura final
fig.update_layout(
    barmode='stack', 
    height=600, 
    width=1000, 
    title_text="Comparative Analysis of Cryptoasset Exchange Volumes by Cryptocurrency and Country",
    title_font=dict(size=20, color="#AA322F"),
    showlegend=True,
    title_x=0.5,  # Centrar el título
    legend_tracegroupgap=180,  # Separar visualmente las leyendas de los subplots
)

# Mostrar la figura
fig.show()


### By type of economy

In [33]:
import pandas as pd

# Supongamos que ya tienes el DataFrame combinado `merged_df` cargado
# Convertir las fechas a formato datetime si aún no está hecho
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])

# Clasificación de monedas fiat en AEs y EMDEs
aes_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CZK']  # Economías Avanzadas
emdes_currencies = ['NGN', 'RUB', 'TRY', 'ZAR', 'UAH', 'BRL', 'ERN', 'PLN', 'RON', 'ARS', 'MXN']  # Mercados Emergentes

# Calcular el volumen en dólares (esto ya debería estar hecho en `merged_df`)
merged_df['volumeUSD'] = merged_df['volume'] * merged_df['PriceUSD']

# Seleccionar solo las columnas necesarias
filtered_df = merged_df[['timestamp', 'fiat_currency', 'crypto_currency', 'volumeUSD']]

# Aplicar el filtro de fechas para incluir solo a partir de 2022
filtered_df = filtered_df[filtered_df['timestamp'] >= '2022-01-01']

# Asegurarse de que todas las criptomonedas estén en mayúsculas
filtered_df['crypto_currency'] = filtered_df['crypto_currency'].str.upper()

# Agrupar todas las criptomonedas que no sean BTC o USDT en "Others"
filtered_df['crypto_currency'] = filtered_df['crypto_currency'].apply(lambda x: x if x in ['BTC', 'USDT'] else 'OTHERS')

# Dividir el DataFrame en AEs y EMDEs
aes_df = filtered_df[filtered_df['fiat_currency'].isin(aes_currencies)]
emdes_df = filtered_df[filtered_df['fiat_currency'].isin(emdes_currencies)]

# Agrupar por mes y 'crypto_currency', y sumar 'volumeUSD' para AEs y EMDEs
aes_grouped = aes_df.groupby([pd.Grouper(key='timestamp', freq='ME'), 'crypto_currency']).sum().reset_index()
emdes_grouped = emdes_df.groupby([pd.Grouper(key='timestamp', freq='ME'), 'crypto_currency']).sum().reset_index()


In [27]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear los subplots uno al lado del otro con un tamaño más adecuado
fig_vol_type_econ = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("AEs", "EMDEs"))

# Definir colores para BTC, USDT y Others
colores = {
    'BTC': '#F7931A',   # Naranja para Bitcoin
    'USDT': '#26A17B',  # Verde para Tether
    'OTHERS': '#B0B0B0'  # Gris para otras criptomonedas
}

# Asegurar el orden de las barras: BTC (primero), USDT (segundo), Others (tercero)
order = ['BTC', 'USDT', 'OTHERS']

# Agregar trazas para AEs
for crypto in order:
    if crypto in aes_grouped['crypto_currency'].unique():
        df_crypto = aes_grouped[aes_grouped['crypto_currency'] == crypto]
        fig_vol_type_econ.add_trace(go.Bar(
            x=df_crypto['timestamp'],
            y=df_crypto['volumeUSD'],
            name=crypto.upper(),
            marker_color=colores.get(crypto.upper(), '#000000'),
            showlegend=True
        ), row=1, col=1)

# Agregar trazas para EMDEs
for crypto in order:
    if crypto in emdes_grouped['crypto_currency'].unique():
        df_crypto = emdes_grouped[emdes_grouped['crypto_currency'] == crypto]
        fig_vol_type_econ.add_trace(go.Bar(
            x=df_crypto['timestamp'],
            y=df_crypto['volumeUSD'],
            name=crypto.upper(),
            marker_color=colores.get(crypto.upper(), '#000000'),
            showlegend=False  # No mostrar leyenda dos veces
        ), row=1, col=2)

# Personalizar el diseño del gráfico para ajustarlo al ancho de la página
fig_vol_type_econ.update_layout(
    title='<b>Cryptoasset Exchange Volumes by Cryptocurrency for AEs and EMDEs<b>',
    title_font_color="#AA322F",    
    xaxis_title='Date',
    yaxis_title='Volume (USD)',
    barmode='stack',
    height=500,  # Ajustar la altura del gráfico para que sea más compacto
    width=800,  # Ajustar el ancho del gráfico para que quepa en la página de Kaggle
    margin=dict(l=20, r=20, t=50, b=20),  # Ajustar los márgenes para maximizar el espacio disponible
    title_font=dict(size=14),  # Reducir el tamaño de la fuente del título para ahorrar espacio
    font=dict(size=10)  # Reducir el tamaño general de la fuente para encajar mejor en la página
)

# Mostrar el gráfico
fig_vol_type_econ.show()


In [45]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filtrar los datos para incluir solo los registros de 2024
aes_grouped_2024 = aes_grouped[aes_grouped['timestamp'].dt.year == 2024]
emdes_grouped_2024 = emdes_grouped[emdes_grouped['timestamp'].dt.year == 2024]

# Sumarizar los volúmenes a nivel anual por criptomoneda
aes_annual = aes_grouped_2024.groupby('crypto_currency')['volumeUSD'].sum().reset_index()
emdes_annual = emdes_grouped_2024.groupby('crypto_currency')['volumeUSD'].sum().reset_index()

# Calcular el volumen total anual para cada tipo de economía
aes_total_annual = aes_annual['volumeUSD'].sum()
emdes_total_annual = emdes_annual['volumeUSD'].sum()

# Calcular los porcentajes de cada criptomoneda en el total anual
aes_annual['percentage'] = aes_annual['volumeUSD'] / aes_total_annual * 100
emdes_annual['percentage'] = emdes_annual['volumeUSD'] / emdes_total_annual * 100

# Crear los subplots uno al lado del otro con un tamaño más adecuado
fig_vol_type_econ = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("AEs", "EMDEs"))

# Definir colores para BTC, USDT y Others
colores = {
    'BTC': '#F7931A',   # Naranja para Bitcoin
    'USDT': '#26A17B',  # Verde para Tether
    'OTHERS': '#B0B0B0'  # Gris para otras criptomonedas
}

# Asegurar el orden de las barras: BTC (primero), USDT (segundo), Others (tercero)
order = ['BTC', 'USDT', 'OTHERS']

# Agregar trazas para AEs
for crypto in order:
    if crypto in aes_annual['crypto_currency'].unique():
        df_crypto = aes_annual[aes_annual['crypto_currency'] == crypto]
        fig_vol_type_econ.add_trace(go.Bar(
            x=['AEs'],
            y=df_crypto['percentage'],
            name=crypto.upper(),
            marker_color=colores.get(crypto.upper(), '#000000'),
            showlegend=True
        ), row=1, col=1)

# Agregar trazas para EMDEs
for crypto in order:
    if crypto in emdes_annual['crypto_currency'].unique():
        df_crypto = emdes_annual[emdes_annual['crypto_currency'] == crypto]
        fig_vol_type_econ.add_trace(go.Bar(
            x=['EMDEs'],
            y=df_crypto['percentage'],
            name=crypto.upper(),
            marker_color=colores.get(crypto.upper(), '#000000'),
            showlegend=False  # No mostrar leyenda dos veces
        ), row=1, col=2)

# Personalizar el diseño del gráfico para ajustarlo al ancho de la página
fig_vol_type_econ.update_layout(
    title='<b>Cryptoasset Exchange Volumes by Cryptocurrency for AEs and EMDEs (2024 Annual) (Percentage)<b>',
    title_font_color="#AA322F",    
    yaxis_title='Percentage of Total Annual Volume',
    barmode='stack',
    height=500,  # Ajustar la altura del gráfico para que sea más compacto
    width=800,  # Ajustar el ancho del gráfico para que quepa en la página de Kaggle
    margin=dict(l=20, r=20, t=50, b=20),  # Ajustar los márgenes para maximizar el espacio disponible
    title_font=dict(size=14),  # Reducir el tamaño de la fuente del título para ahorrar espacio
    font=dict(size=10)  # Reducir el tamaño general de la fuente para encajar mejor en la página
)

# Mostrar el gráfico
fig_vol_type_econ.show()


## Appendix: Automating Graph Exports

This appendix covers the automation of exporting graphs to different platforms. It includes two main subsections: one for publishing graphs on GitHub and another (to be added) for exporting to Plotly Studio Chart.

### Subsection 1: Publishing Graphs on Plotty

In [ ]:
!pip install chart-studio --quiet

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")

In [ ]:
import chart_studio.plotly as py
import chart_studio.tools as tls

# Configurar las credenciales de Plotly
secret_key = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")
tls.set_credentials_file(username='crypto-financial-stability-monitor', api_key=secret_key)

# Exportar los gráficos con nombres descriptivos
py.plot(fig, filename='crypto-network', auto_open=False)
py.plot(fig_crypto_count_per_fiat, filename='crypto-count-per-fiat', auto_open=False)
py.plot(fig_btc_usdt_pairs, filename='btc-usdt-pairs-per-fiat', auto_open=False)
py.plot(fig_unique_frequency, filename='unique-frequency-cryptocurrencies', auto_open=False)
py.plot(fig_heatmap_animated, filename='animated-heatmap-crypto-fiat-over-time', auto_open=False)
py.plot(fig_vol_countries, filename='crypto-volumes-by-country', auto_open=False)
py.plot(fig_vol_cryptos, filename='crypto-volumes-by-cryptocurrency', auto_open=False)
py.plot(fig_vol_type_econ, filename='crypto-volumes-by-economy-type', auto_open=False)

### Subsection 2: Publishing Graphs on GitHub

This subsection outlines the steps to automate the export of your Plotly graphs to HTML files and publish them to a GitHub repository.

#### Steps:

1. **Export Graphs to HTML**: Save the generated graphs as HTML files.
2. **GitHub Setup**: Configure GitHub credentials and clone the repository.
3. **Move Files to Repository**: Copy the HTML files to the cloned repository.
4. **Commit and Push**: Commit and push the files to GitHub.


In [ ]:
import os
import git
from kaggle_secrets import UserSecretsClient

# Retrieve GitHub token securely from Kaggle secrets
user_secrets = UserSecretsClient()
GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")

# Set up the repository URL using the secured token
REPO_URL = "https://{}@github.com/emigiupponi/CryptoFiatVisualizer.git".format(GITHUB_TOKEN)

# Clone the repository if not already done
if not os.path.exists('repo'):
    repo = git.Repo.clone_from(REPO_URL, 'repo')

repo = git.Repo('repo')

# Export graphs to HTML
fig1.write_html("fig_heatmap_animated.html")

# Move HTML files to the repo
os.system('cp crypto_fiat_animation.html repo/')

# Commit and push to GitHub
repo.index.add(['crypto_fiat_animation.html'])
repo.index.commit("Add crypto_fiat animations")
repo.remote(name='origin').push()
